In [1]:
%pip install huggingface_hub


     |████████████████████████████████| 447 kB 7.0 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=(ENV.Secret)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
topics = [
    "Abortion",
    "Agriculture",
    "Animal Rights",
    "Appropriations",
    "Athletics and Sports",
    "Banking and Financial Services",
    "Border Security",
    "Budget",
    "Campaign Finance Reform",
    "Child Abuse",
    "Child Welfare and Health",
    "Civil Liberties/Privacy",
    "Climate Change",
    "Commerce",
    "Community/Economic Development",
    "Consumer Protection",
    "COVID Relief",
    "Debt Ceiling",
    "Defense and Military",
    "Disabilities",
    "Education",
    "Election Security",
    "Energy",
    "Environment",
    "Ethics and Rules",
    "Foreign Affairs",
    "Guns",
    "Healthcare",
    "Homeland Security",
    "Housing",
    "Immigration",
    "Intelligence",
    "Interior",
    "Judges",
    "Judiciary",
    "Labor",
    "Law Enforcement",
    "Native Americans",
    "Nutrition",
    "Postal Service",
    "Refugees",
    "Science and Innovation",
    "Senior Citizens",
    "Social Security and Retirement",
    "Surveillance",
    "Taxes",
    "Technology",
    "Trade",
    "Transportation",
    "Veterans",
    "Voting Rights"
]

In [ ]:
### JSON Object Format
# {
    # - prefix
    # - first_name
    # - last_name
    # - street_address
    # - city
    # - state
    # - zip_code
    # - email
    # - phone_number
    # - message_subject
    # - message_body# }


In [61]:
def process_email(email_json):

    email_content = email_json["message_body"]
    email_subject = email_json["message_subject"]
    sender_city = email_json["city"]
    sender_state = email_json["state"]

    subject_prompt = [
        {
            "role": "user",
            #"content": f"Can you classify this email into exactly one of the following categories (the one that is most relevant to the content of the email): {', '.join(topics)}. "
                       #f"Only provide the topic name as a response. "
                       #f"Email content: '{email_content}'"
            "content": f"Read the following email and identify the main subjects or topics exactly as one of the following categories: {', '.join(topics)}. "
                       f"Only provide the topic name as a response. "
                       f"Email subject: '{email_subject}'"
                       f"Email content: '{email_content}'"
        }
    ]

    classification = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=subject_prompt,
        temperature=0.2,
        max_tokens=10,
        top_p=0.9,
        stream=False
    )

    subjects = classification.choices[0].message["content"].strip()
    print("Identified Subject:", subjects)

    sentiment_prompt = [
        {
            "role": "user",
            "content": f"You just read this email: '{email_content}' and identified the main subjects to be to be '{subjects}'."
                       f"Given that the subject of the email is '{subjects}', analyze the overall sentiment of the email content as positive high, positive medium, positive low, neutral, negative low, negative medium, negative high."
                       f"Only provide 'Positive High', 'Positive Medium', 'Positive Low', 'Neutral', 'Negative Low', 'Negative Medium', 'Negative High' as a response. No other information is needed"
        }
    ]

    sentiment_analysis = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=sentiment_prompt,
        temperature=0.2,
        max_tokens=10,
        top_p=0.9,
        stream=False
    )

    sentiment = sentiment_analysis.choices[0].message["content"].strip()
    print("Identified Sentiment:", sentiment)

    subtopic_prompt = [
        {
            "role": "user",
            "content": f"You have read the email: '{email_content}' and identified the subject as '{subjects}'. "
                       f"Generate a single concise sentence (maximum 15 words) that captures a specific subtopic from the email content. "
                       f"Your response should be only one sentence, without any additional options or explanations. "
                       f"Example Email: 'Dear Congressman, I am concerned about a nearby farm dumping waste into the Truckee River, harming wildlife and our community.' "
                       f"Identified Subjects: Agriculture, Environment. "
                       f"Expected Output: 'Negative impact of farm waste on local waterways.'"
        }
    ]

    subtopic_analysis = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=subtopic_prompt,
        temperature=0.2,
        max_tokens=50,
        top_p=0.9,
        stream=False
    )

    subtopic = subtopic_analysis.choices[0].message["content"].strip()
    print("Identified Subtopic:", subtopic)

    # sentiment_score_prompt = [
    #     {
    #         "role": "user",
    #         #"content": f"You identified the sentiment to be '{sentiment}'. Given that, what sentiment score, from a range of -1 to 1, would you give?"
    #                    #f"For guidance, very negative would be in the range of -1 to -0.6, negative would be in the range of -0.6 to -0.2, neutral would be in the range of -0.2 to 0.2, positive would be in the range of 0.2 to 0.6, and very positive would be in the range of 0.6 to 1."
    #                    #f"Only provide the number as a response. No other explanation, introduction, or anything else is required."

    #         "content": #f"In the sentiment scale I am using, very negative would be in the range of -1 to -0.6, negative would be in the range of -0.6 to -0.2, neutral would be in the range of -0.2 to 0.2, positive would be in the range of 0.2 to 0.6, and very positive would be in the range of 0.6 to 1."
    #                    f"You just read this email: '{email_content}' and identified the subject to be '{subject}'."
    #                    f"Then, you identified the sentiment to be '{sentiment}'. "
    #                    "Given all of this, what sentiment score, from a range of -1 to 1, would you give?"
    #                    "In the sentiment scale I am using, 'Very Negative' would be in the range of -1 to -0.6, 'Negative' would be in the range of -0.6 to -0.2, 'Neutral' would be in the range of -0.2 to 0.2, 'Positive' would be in the range of 0.2 to 0.6, and 'Very Positive' would be in the range of 0.6 to 1. Make sure that the sentiment you identified and the score you assign match the ranges I provided."
    #                    f"Only provide the number as a response. No other explanation, introduction, or anything else is required."
    #     }
    # ]

    # sentiment_score_analysis = client.chat.completions.create(
    #     model="meta-llama/Llama-3.2-3B-Instruct",
    #     messages=sentiment_score_prompt,
    #     temperature=0.2,
    #     max_tokens=5,
    #     top_p=0.9,
    #     stream=False
    # )

    # sentiment_score = sentiment_score_analysis.choices[0].message["content"].strip()
    # print("Identified Sentiment Score:", sentiment_score)

    email_json["topic"] = subjects
    email_json["sentiment"] = sentiment
    email_json["subtopic"] = subtopic
    return email_json


In [67]:
# trade
# Very Negative
email_content = "Dear Congressman Johnson, I am writing to express my deep concern and disappointment regarding the recent trade agreements your administration has been negotiating with our country's largest trading partners. As a small business owner and a constituent of your district, I am appalled by the lack of transparency and consultation with stakeholders in the process. The recent tariffs imposed on our agricultural products have already had a devastating impact on my family's farm, causing significant losses and uncertainty for our future. I am not alone in my frustration, as numerous farmers and business owners in our district have reached out to me expressing similar concerns. Furthermore, I am troubled by the apparent disregard for the long-term consequences of these agreements, which will likely lead to a decline in our country's manufacturing sector and job losses. As a constituent, I expect more from my elected representatives, particularly when it comes to issues that directly affect our local economy. I urge you to reconsider your stance on these trade agreements and work towards finding a more balanced and fair solution that prioritizes the interests of American workers and businesses. I would appreciate the opportunity to discuss this matter further with you and explore alternative solutions that can benefit our community. Sincerely, Emily Wilson"
email_json = {  
    "prefix": "Ms.",
    "first_name": "Emily",
    "last_name": "Wilson",
    "street_address": "123 Main St",
    "city": "San Francisco",
    "state": "CA",
    "zip_code": "94101",
    "email": "emily.wilson@example.com",
    "phone_number": "555-123-4567",
    "message_body": email_content,
    "message_subject": "Dissapointment in recent trade agreements",
}
classification = process_email(email_json)
classification

Identified Subject: trade
Identified Sentiment: Negative High
Identified Subtopic: Lack of transparency and consultation in trade agreement negotiations with major trading partners.


{'prefix': 'Ms.',
 'first_name': 'Emily',
 'last_name': 'Wilson',
 'street_address': '123 Main St',
 'city': 'San Francisco',
 'state': 'CA',
 'zip_code': '94101',
 'email': 'emily.wilson@example.com',
 'phone_number': '555-123-4567',
 'message_body': "Dear Congressman Johnson, I am writing to express my deep concern and disappointment regarding the recent trade agreements your administration has been negotiating with our country's largest trading partners. As a small business owner and a constituent of your district, I am appalled by the lack of transparency and consultation with stakeholders in the process. The recent tariffs imposed on our agricultural products have already had a devastating impact on my family's farm, causing significant losses and uncertainty for our future. I am not alone in my frustration, as numerous farmers and business owners in our district have reached out to me expressing similar concerns. Furthermore, I am troubled by the apparent disregard for the long-t

In [66]:
# guns
# Very Positive
email_content = "Hey Congressman, I just wanted to reach out and say thanks for all the work you've been doing on gun control. I know it's a tough issue, but I really appreciate the effort you're putting in. My cousin was in a situation where he had to use his gun to protect himself and his family, and it was a real wake-up call for him. He's been thinking a lot about the Second Amendment and how it applies to our community, and he's come to realize that we need to be doing more to prevent gun violence. I know you're not going to agree with everything, but I think you're on the right track with some of the bills you've been proposing. I'd love to hear more about your thoughts on the matter and how we can work together to make a difference. My kids are getting older and they're starting to ask more questions about guns and gun safety, and I want to make sure they're getting the right information. Thanks again for all your hard work, and I look forward to hearing from you soon. Best, Sarah"
email_json = {
    "prefix": "Ms.",
    "first_name": "Sarah",
    "last_name": "Johnson",
    "street_address": "123 Main St",
    "city": "San Francisco",
    "state": "CA",
    "zip_code": "94101",
    "email": "sarah.johnson@example.com",
    "phone_number": "555-123-4567",
    "message_body": email_content,
    "message_subject": "Thank you for your work!"        
}
classification = process_email(email_json)
classification

Identified Subject: guns
Identified Sentiment: Positive High
Identified Subtopic: Application of the Second Amendment to community gun safety concerns.


{'prefix': 'Ms.',
 'first_name': 'Sarah',
 'last_name': 'Johnson',
 'street_address': '123 Main St',
 'city': 'San Francisco',
 'state': 'CA',
 'zip_code': '94101',
 'email': 'sarah.johnson@example.com',
 'phone_number': '555-123-4567',
 'message_body': "Hey Congressman, I just wanted to reach out and say thanks for all the work you've been doing on gun control. I know it's a tough issue, but I really appreciate the effort you're putting in. My cousin was in a situation where he had to use his gun to protect himself and his family, and it was a real wake-up call for him. He's been thinking a lot about the Second Amendment and how it applies to our community, and he's come to realize that we need to be doing more to prevent gun violence. I know you're not going to agree with everything, but I think you're on the right track with some of the bills you've been proposing. I'd love to hear more about your thoughts on the matter and how we can work together to make a difference. My kids are 

In [ ]:
import supabase
from datetime import datetime
url = ""
key = ""

supabase = supabase.create_client(url, key)


In [1]:

    
def find_sender_id(email):
    response = supabase.table('people').select('*').eq('email', email).execute()
    if response.data:
        return response.data[0]['id']
    else:
        return None
    
def add_sender_to_db(sender_json):
    supabase.table('people').insert(sender_json).execute()

def add_email_to_db(email_json):
    response = supabase.table('received_emails').insert(email_json).execute()

def find_congressperson_id(email):
    return None

def find_congressional_district(email):
    return None

def find_topic_id(topic):
    response = supabase.table('topics').select('*').eq('name', topic).execute()
    return response.data[0]['id']

def add_to_db(proecessed_email_json):
    sender_id = find_sender_id(proecessed_email_json["email"])
    if sender_id is None:
        sender_json = {
            "prefix": proecessed_email_json["prefix"],
            "first_name": proecessed_email_json["first_name"],
            "last_name": proecessed_email_json["last_name"],
            "email": proecessed_email_json["email"],
            "address": proecessed_email_json["street_address"],
            "city": proecessed_email_json["city"],
            "state": proecessed_email_json["state"],
            "zip_code": proecessed_email_json["zip_code"],
            "phone": proecessed_email_json["phone_number"],
            "updated_at": datetime.now(),
            "congressional_district": find_congressional_district(proecessed_email_json["email"])
        }
        add_sender_to_db(sender_json)
        sender_id = find_sender_id(proecessed_email_json["email"])

    topic_id = find_topic_id(proecessed_email_json["topic"])
    received_email_json = {
        "constituent_id": sender_id,
        "representative_id": None,
        "subject": proecessed_email_json["message_subject"],
        "message": proecessed_email_json["message_body"],
        "sentiment": proecessed_email_json["sentiment"],
        "subtopic": proecessed_email_json["subtopic"],
        "topic_id": topic_id,
        "timestamp": datetime.now()
    }
    add_email_to_db(received_email_json)
    



In [2]:
def process_form_json(form_json):
    processed_email_json = process_email(form_json)
    add_to_db(processed_email_json)

